In [ ]:
%pip install "pydantic-ai[logfire]" "nest_asyncio"

In [ ]:
from agents import Agent, Runner

import logfire
# configure Logfire for console only
logfire.configure(
    service_name="my_agent_service",
    send_to_logfire=False,
    inspect_arguments=True
)
logfire.instrument_openai_agents()

agent = Agent(
    name="Research Assistant",
    instructions="You are a helpful Research Assistant which helps professors in various tasks.",
)

result = await Runner.run(agent, "Get me the abstract of arXiv:1706.03762")
print(result.final_output)

06:17:25.969 OpenAI Agents trace: Agent workflow
06:17:25.969   Agent run: 'Research Assistant'
06:17:25.970     Responses API with 'gpt-4o'
The paper with the identifier arXiv:1706.03762 is titled "Attention Is All You Need." Here is the abstract:

"The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-

In [35]:
import requests
from bs4 import BeautifulSoup
from typing import Dict, Any
from agents import function_tool


@function_tool
def fetch_paper(arxiv_id: str) -> Dict[str, Any]:
    """
    Fetches paper information from arXiv given its ID.
    
    Args:
        arxiv_id: The arXiv ID (e.g., '1706.03762')
    
    Returns:
        A dictionary containing paper information including:
        - title
        - authors
        - abstract
        - pdf_url
        - published_date
    """
    # Clean the input ID
    arxiv_id = arxiv_id.strip().replace("arXiv:", "")
    
    # Construct the URL
    url = f"https://arxiv.org/abs/{arxiv_id}"
    
    try:
        # Fetch the page
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract paper information
        title = soup.select_one("h1.title").text.replace("Title:", "").strip()
        authors = [author.strip() for author in soup.select_one("div.authors").text.replace("Authors:", "").split(",")]
        abstract = soup.select_one("blockquote.abstract").text.replace("Abstract:", "").strip()
        pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
        
        # Try to extract published date
        date_element = soup.select_one("div.dateline")
        published_date = date_element.text.strip() if date_element else "Date not found"
        
        return {
            "title": title,
            "authors": authors,
            "abstract": abstract,
            "pdf_url": pdf_url,
            "published_date": published_date,
            "arxiv_id": arxiv_id
        }
        
    except Exception as e:
        print(f"Error fetching paper: {e}")
        return {
            "error": str(e),
            "arxiv_id": arxiv_id
        }


agent_1 = Agent(
    name="Research Assistant",
    instructions="You are a helpful Research Assistant which helps professors in various tasks.",
    tools=[fetch_paper]
)

result = await Runner.run(agent_1, "Get me the abstract of arXiv:1706.03762")
print(result.final_output)

06:17:31.294 OpenAI Agents trace: Agent workflow
06:17:31.295   Agent run: 'Research Assistant'
06:17:31.297     Responses API with 'gpt-4o'
06:17:32.132     Function: fetch_paper
06:17:32.188     Responses API with 'gpt-4o'
The abstract for the paper titled "Attention Is All You Need" (arXiv:1706.03762) is as follows:

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best r

In [20]:
result = await Runner.run(agent_1, "Get me the abstract of arXiv:1706.03762 and tell me where each author is working currently")
print(result.final_output)

06:04:36.688 OpenAI Agents trace: Agent workflow
06:04:36.688   Agent run: 'Research Assistant'
06:04:36.689     Responses API with 'gpt-4o'
06:04:37.627     Function: fetch_paper
06:04:37.684     Responses API with 'gpt-4o'
06:04:38.916     Function: fetch_paper
06:04:38.972     Responses API with 'gpt-4o'
### Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best 

In [41]:
import logging
from agents import WebSearchTool


# Set logging level to debug
logging.basicConfig(level=logging.DEBUG)

agent_2 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper],
)

result = await Runner.run(
    agent_2,
    "Get me the abstract of arXiv:1706.03762 and tell me where each author is working currently.")
print(result.final_output)

DEBUG:openai.agents:Creating trace Agent workflow with id trace_33433db04de2478aaf2e76dde17f8025
DEBUG:openai.agents:Setting current trace: trace_33433db04de2478aaf2e76dde17f8025
DEBUG:openai.agents:Creating span <agents.tracing.span_data.AgentSpanData object at 0x11ea14cb0> with id None
DEBUG:openai.agents:Running agent Research Assistant (turn 1)
DEBUG:openai.agents:Creating span <agents.tracing.span_data.ResponseSpanData object at 0x11c0b6080> with id None
DEBUG:openai.agents:Calling LLM gpt-4o with input:
[
  {
    "content": "Get me the abstract of arXiv:1706.03762 and tell me where each author is working currently.",
    "role": "user"
  }
]
Tools:
[
  {
    "type": "web_search_preview",
    "user_location": null,
    "search_context_size": "medium"
  },
  {
    "name": "fetch_paper",
    "parameters": {
      "properties": {
        "arxiv_id": {
          "description": "The arXiv ID (e.g., '1706.03762')",
          "title": "Arxiv Id",
          "type": "string"
        }
    

06:29:33.990 OpenAI Agents trace: Agent workflow
06:29:33.991   Agent run: 'Research Assistant'
06:29:33.995     Responses API with 'gpt-4o'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Apr 2025 06:29:34 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-version', b'2020-10-01'), (b'openai-organization', b'user-thoiewvv8uw64smqtyk9d229'), (b'x-request-id', b'req_37e23106ebdd65def9ab421c09cf6759'), (b'openai-processing-ms', b'652'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'933296880c86df86-SEA'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBU

06:29:34.900     Function: fetch_paper
06:29:34.951     Responses API with 'gpt-4o'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Apr 2025 06:29:54 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-version', b'2020-10-01'), (b'openai-organization', b'user-thoiewvv8uw64smqtyk9d229'), (b'x-request-id', b'req_f9a07f9c801ad17a2f979f5ac74b9d5b'), (b'openai-processing-ms', b'19686'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9332968dc8a7df86-SEA'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DE

The paper titled "Attention Is All You Need" introduces the Transformer, a novel network architecture based solely on attention mechanisms, eliminating the need for recurrence and convolutions. This design enhances parallelization and reduces training time. The Transformer achieved superior performance in machine translation tasks, notably obtaining a BLEU score of 28.4 on the WMT 2014 English-to-German translation task and 41.8 on the English-to-French task. Additionally, the model demonstrated effective generalization to other tasks, such as English constituency parsing.

Regarding the current affiliations of the authors:

- **Ashish Vaswani**: Co-founder and CEO of Essential AI since 2022.

- **Noam Shazeer**: CEO of Character AI, Palo Alto, California, USA.

- **Niki Parmar**: Co-founder of Essential AI.

- **Jakob Uszkoreit**: Co-founder and CEO of Inceptive since 2021.

- **Llion Jones**: Co-founder at Sakana AI since August 2023. ([theorg.com](https://theorg.com/org/sakana-ai/or

In [42]:

from agents import WebSearchTool


agent_2 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper],
)

result = await Runner.run(
    agent_2,
    "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic")
print(result.final_output)

DEBUG:openai.agents:Creating trace Agent workflow with id trace_9b2e5cf3e03844bb9e448951f3433878
DEBUG:openai.agents:Setting current trace: trace_9b2e5cf3e03844bb9e448951f3433878
DEBUG:openai.agents:Creating span <agents.tracing.span_data.AgentSpanData object at 0x11e727890> with id None
DEBUG:openai.agents:Running agent Research Assistant (turn 1)
DEBUG:openai.agents:Creating span <agents.tracing.span_data.ResponseSpanData object at 0x11e024960> with id None
DEBUG:openai.agents:Calling LLM gpt-4o with input:
[
  {
    "content": "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic",
    "role": "user"
  }
]
Tools:
[
  {
    "type": "web_search_preview",
    "user_location": null,
    "search_context_size": "medium"
  },
  {
    "name": "fetch_paper",
    "parameters": {
      "properties": {
        "arxiv_id": {
          "description": "The arXiv ID (e.g., '1706.03762')",
          "title": "Arxiv 

06:36:22.749 OpenAI Agents trace: Agent workflow
06:36:22.750   Agent run: 'Research Assistant'
06:36:22.752     Responses API with 'gpt-4o'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Apr 2025 06:36:24 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-version', b'2020-10-01'), (b'openai-organization', b'user-thoiewvv8uw64smqtyk9d229'), (b'x-request-id', b'req_271577e533f15b6429422d421d2e032d'), (b'openai-processing-ms', b'1161'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=V5Rn84r1xq.53PrfifZw7VD5eB6ssSaWFrUuJ94CuA4-1745130984-1.0.1.1-XW21auRTUBRtPG53lwAwh6PXap4FwIKimSVMiekQhvayrfFCDhh1OZljbJHuNorGhg9pYwAVHtZqAVaJTdWV0_CqYzOKjgytnhyUgA8ZURA; path=/; expires=Sun, 20-Apr-25 07:06:24 GMT; domain=.api.openai.com; HttpOnly; Secure; SameSite=None'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9332a082ced87664-SEA'), (b'Content-Encoding', b'gzip'), (b'

I currently don't have the ability to view or access files directly. If you could provide the text or main points of your draft proposal here, I'd be happy to help you with feedback based on current information.


In [44]:
from typing import Annotated
from agents import function_tool


@function_tool
async def read_local_file(
    file_path: Annotated[str, "Path to the file to read"]
) -> str:
    """
    Reads the content of a local file and returns it as a string.
    
    Args:
        file_path: Path to the file to read
        
    Returns:
        The content of the file as a string
        
    Raises:
        FileNotFoundError: If the file does not exist
        PermissionError: If the file cannot be read due to permission issues
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except PermissionError:
        return f"Error: Permission denied when trying to read '{file_path}'."
    except Exception as e:
        return f"Error reading file: {str(e)}"


agent_3 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper, read_local_file],
)

result = await Runner.run(
    agent_3,
    "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic")
print(result.final_output)

DEBUG:openai.agents:Creating trace Agent workflow with id trace_6fe72e8e53b147ffb0638cf15b855880
DEBUG:openai.agents:Setting current trace: trace_6fe72e8e53b147ffb0638cf15b855880
DEBUG:openai.agents:Creating span <agents.tracing.span_data.AgentSpanData object at 0x11e064cb0> with id None
DEBUG:openai.agents:Running agent Research Assistant (turn 1)
DEBUG:openai.agents:Creating span <agents.tracing.span_data.ResponseSpanData object at 0x11e027de0> with id None
DEBUG:openai.agents:Calling LLM gpt-4o with input:
[
  {
    "content": "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic",
    "role": "user"
  }
]
Tools:
[
  {
    "type": "web_search_preview",
    "user_location": null,
    "search_context_size": "medium"
  },
  {
    "name": "fetch_paper",
    "parameters": {
      "properties": {
        "arxiv_id": {
          "description": "The arXiv ID (e.g., '1706.03762')",
          "title": "Arxiv 

06:38:22.260 OpenAI Agents trace: Agent workflow
06:38:22.261   Agent run: 'Research Assistant'
06:38:22.263     Responses API with 'gpt-4o'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Apr 2025 06:38:23 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-version', b'2020-10-01'), (b'openai-organization', b'user-thoiewvv8uw64smqtyk9d229'), (b'x-request-id', b'req_ce1213ec9ee5458afa27e29c6dc1d6b8'), (b'openai-processing-ms', b'648'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9332a36dba1f2849-SEA'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBU

06:38:23.101     Function: read_local_file
06:38:23.105     Responses API with 'gpt-4o'


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 20 Apr 2025 06:38:31 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-version', b'2020-10-01'), (b'openai-organization', b'user-thoiewvv8uw64smqtyk9d229'), (b'x-request-id', b'req_0c0a8dd906a6adaf06e597b6850a0ea7'), (b'openai-processing-ms', b'8157'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'cf-cache-status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'9332a372cd992849-SEA'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEB

Your draft proposal, "Adaptive Context Management for Large Language Models: Optimizing Information Retrieval and Utilization in Multi-Agent Systems," presents a compelling approach to enhancing context management in large language models (LLMs) within multi-agent systems. To strengthen your proposal, consider the following feedback based on recent research:

**1. Incorporate Recent Surveys and Frameworks:**
A comprehensive survey titled "A Survey on Context-Aware Multi-Agent Systems: Techniques, Challenges and Future Directions" by Hung Du et al. (2024) provides an in-depth analysis of integrating context-aware systems with multi-agent systems. This work outlines a general process model applicable across various domains, which could serve as a foundational reference for your framework. ([arxiv.org](https://arxiv.org/abs/2402.01968v1?utm_source=openai))

**2. Address Identified Challenges:**
The aforementioned survey highlights several challenges in context-aware multi-agent systems, s